In [ ]:
import re
import pandas as pd

In [ ]:
data = pd.read_csv("data/city.csv")
data['Postcode'] = data['Postcode'].astype('int64')
data['Car'] = data['Car'].astype('int64')
data['Bedroom'] = data['Bedroom'].astype('int64')
data['Bathroom'] = data['Bathroom'].astype('int64')
data['Propertycount'] = data['Propertycount'].astype('int64')
data['YearBuilt'] = data['YearBuilt'].astype('int64')
train = data.copy()

In [ ]:
train = train.drop(["index", "Coordinates"], axis=1)

total_rooms = train["Bathroom"] + train["Bedroom"] + train["Rooms"]
train["MeanRoomsSquare"] = train["BuildingArea"] / total_rooms

area_diff = train["BuildingArea"] - train["Landsize"]
area_total = train["BuildingArea"] + train["Landsize"]
train["AreaRatio"] = area_diff / area_total

train["Date"] = pd.to_datetime(train["Date"])
train["MonthSale"] = train["Date"].dt.month
train["WeekdaySale"] = train["Date"].dt.weekday
train["Weekend"] = train["WeekdaySale"].apply(lambda cell: 1 if cell in [5, 6] else 0)

train["AgeBuilding"] = train['Date'].dt.year - train['YearBuilt']
train = train.drop("YearBuilt", axis=1)

common_sellers = train["SellerG"].value_counts().nlargest(49).index
train["SellerG"] = train["SellerG"].apply(lambda cell: cell if cell in common_sellers else "other")

popular_subtype = train["Suburb"].value_counts().nlargest(119).index
train["Suburb"] = train["Suburb"].apply(lambda cell: cell if cell in popular_subtype else "other")

def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = train['Address'].apply(get_street_type)
popular_stypes = street_types.value_counts().nlargest(10).index
train['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')

train = train.drop('Address', axis=1)

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in train.columns: # цикл по именам столбцов
    if train[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        train[col] = train[col].astype('category') # преобразуем тип столбца
        
train['Type'] = train['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})

train.info()

# МЕТОД SORT_VALUES()

In [ ]:
mask1 = train['AreaRatio'] < -0.8
mask2 = train['Type'] == 'townhouse'
mask3 = train['SellerG'] == 'McGrath'
train[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [ ]:
train.sort_values(by="AreaRatio",ignore_index=True, ascending=True).iloc[1557]

mask1 = train["Type"] == "townhouse"
mask2 = train["Rooms"] > 2

train[mask1 & mask2].sort_values(
    by=["Rooms", "MeanRoomsSquare"],
    ascending=[True, False],
    ignore_index=True
).iloc[18]

# МЕТОД GROUPBY()

In [ ]:
train.groupby(by='Type').mean()

train.groupby(by='Type')['Price'].mean()

train.groupby(by='Regionname')['Distance'].min().sort_values(ascending=False)

train.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

train.groupby('MonthSale')['Price'].agg('describe')

train.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

In [ ]:
train.groupby(by="Rooms")["Price"].agg(
    ["mean"]
).sort_values(by="mean", ascending=False)

train.groupby(by="Regionname")["Lattitude"].agg(
    ["std"]    
).sort_values(by="std", ascending=True)

mask1 = pd.to_datetime("2017-05-01") < train["Date"] 
mask2 = train["Date"] <= pd.to_datetime("2017-09-01")
train[mask1 & mask2].groupby(by="SellerG")["Price"].agg(
    ["sum"]
).sort_values(by="sum", ascending=True)

train.groupby(['Rooms'])[['Price', 'BuildingArea']].median()

train.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

# МЕТОД PIVOT_TABLE() ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

In [ ]:
train.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type'
).round()

train.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

pivot = train.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

train.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

In [ ]:
pivot.columns
display(pivot['mean']['unit'])

mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

In [ ]:
train.pivot_table(
    values="Price",
    index="SellerG",
    columns="Type",
    aggfunc="mean",
    fill_value=0
).sort_values(by="unit", ascending=False)

train.pivot_table(
    values="BuildingArea",
    index="Type",
    columns="Rooms",
    aggfunc="median",
    fill_value=0
)

---

In [ ]:
ratings1 = pd.read_csv("data/movies_data/ratings1.csv")
ratings2 = pd.read_csv("data/movies_data/ratings2.csv")
movies = pd.read_csv("data/movies_data/movies.csv")
dates = pd.read_csv("data/movies_data/dates.csv")

In [ ]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

ratings = ratings.drop_duplicates(ignore_index=True)

print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

ratings_dates = pd.concat([ratings, dates], axis=1)


# МЕТОД JOIN()

In [ ]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined)

# МЕТОД MERGE()

In [ ]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged)

merge_ratings = ratings1.merge(ratings2, how='outer')
display(merge_ratings)

In [ ]:

items_df = pd.DataFrame({
'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged = purchase_df.merge(items_df, on="item_id", how="inner")

merged['total'] = merged["price"] * merged["stock_count"]
income = merged["total"].sum()

print(merged)

![img](https://lms.skillfactory.ru/assets/courseware/v1/dd977e42966f3bb21bd5cdd27f91ba39/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-u1-md12_7_10.png)

# ПОВТОРЕНИЕ

In [ ]:
def get_year_release(arg):
    candidates = re.findall(r'\(\d{4}\)', arg) 
    if len(candidates) == 0:
        return None
    year = candidates[0].replace('(', '')
    year = year.replace(')', '')
    return int(year)

merged['year_release'] = merged["title"].apply(get_year_release)

merged['year_release'].isnull().sum()

#merged[merged["year_release"] == 1999].sort_values(by="rating")
merged[merged["year_release"] == 1999].groupby(by="title")['rating'].agg('mean').sort_values()

merged[merged["year_release"] == 2010].groupby(by="genres")['rating'].agg("mean").sort_values()

merged.groupby(by="userId")["genres"].agg(
    ["nunique"]
).sort_values(by="nunique")

merged.groupby(by="userId")["rating"].agg(
    ["count" ,"mean"]
).sort_values(by=["count", "mean"], ascending=[True, False])

merged[merged["year_release"] == 2018].groupby(by="genres")["rating"].agg(
    ["mean", "count"]
)

merged["year_rating"] = pd.to_datetime(merged["date"]).dt.year
merged.pivot_table(
    values="rating",
    index="year_rating",
    columns="genres",
    aggfunc="mean",
    fill_value=0
)

In [ ]:
orders = pd.read_csv("data/orders_and_products/orders.csv", sep=";")
products = pd.read_csv("data/orders_and_products/products.csv", sep=";")

orders_products = orders.merge(products, left_on="ID товара", right_on="Product_ID", how="left")
orders_products